In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Automat_EAD/GT_EAD"

/content/drive/MyDrive/Automat_EAD/GT_EAD


In [ ]:
!pip install -q torch
!pip install -q transformers
!pip install -q bitsandbytes
!pip install -q accelerate
!pip install -q peft
!pip install -q trl
!pip install -q sentencepiece
!pip install -q tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00


In [ ]:
from time import time
import os
import torch
import sentencepiece
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    LlamaTokenizerFast,
    TrainingArguments,
    pipeline,
    Trainer
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Nous téléchargeons le modèle (à n'executer qu'une fois)

In [ ]:
!mkdir -p llama-2-7b-hf

!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/config.json
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/generation_config.json
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/model-00001-of-00002.safetensors
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/model-00002-of-00002.safetensors
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/model.safetensors.index.json
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/pytorch_model-00001-of-00003.bin
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/pytorch_model-00002-of-00003.bin
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/pytorch_model-00003-of-00003.bin
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/pytorch_model.bin.index.json
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/special_tokens_map.json
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/tokenizer.json
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/tokenizer.model
!wget https://huggingface.co/NousResearch/Llama-2-7b-hf/blob/main/tokenizer_config.json

Pour plus de commodité, nous déposons l'ensemble des paramètres ici :

---



In [ ]:
# Les paramètres de base
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

per_device_train_batch_size = 4 #nombre d'extraits pour chaque fichier
gradient_accumulation_steps = 4 #accumulation des gradients sur 10 mini-lots, ce qui revient à un per_device_train_batch_size virtuel de 4
#automatise le processus d'abaissement de la taille du lot
#fonctionne avec une décroissance exponentielle, réduisant la taille du lot de moitié après chaque échec d'exécution
#Le per_device_train_batch_size est utilisé comme taille de lot initiale pour commencer. Ainsi, si vous utilisez la valeur par défaut de 8, il démarre l'entraînement avec une taille de lot de 8(sur un seul appareil), et en cas d'échec, il redémarrera la procédure d'entraînement avec une taille de lot de 4
auto_find_batch_size=False

learning_rate = 2e-5 #Le taux d'apprentissage. Il est recommandé de le garder plutôt élevé pour des corpus en français et pour des "petits" LLMs.
max_grad_norm = 0.3
max_seq_length = 512 # the max number of tokens allowed for each sample

# Sauvegarde des steps (permet de faire redémarrer l'entraînement si le fine-tuning ne fonctionne pas)
# Ça prend aussi de l'espace sur colab. 100 est généralement un bon compromis (au pire on ne perd que 10-20 minutes d'entraînement)
save_steps = 100

# Nombre de steps : à ajuster selon la taille du corpus et le nombre d'epochs à faire tourner.
#Il est aussi possible de définir un nombre d'epochs mais je préfère travailler par steps pour avoir une meilleure estimation du temps nécessaire.
max_steps = 1

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Log every X updates steps
logging_steps = 1

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "constant"

# Optimizer to use, original is paged_adamw_8bit
optim = "paged_adamw_32bit"

# Enable fp16 training
fp16 = True

# Enable bf16 training
bf16 = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Nous activons la quantisation en 4-bit pour le fine-tuning. Cela accélère significativement l'entraînement avec un impact limité sur le résultat final.
use_4bit = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activer la quantification imbriquée pour les modèles de base 4 bits
use_nested_quant = False

# Le modèle de base que nous allons utiliser.
model_name = "llama-2-7b-hf"

# Le nom du nouveau modèle
new_model_name = "llama-2-7b-ead-v5"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Le dossier final où seront déposés les poids du modèle.
output_dir = "./llama-2-7b-ead-v5"

# Use packing dataset creating
packing = False

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./llama-2-7b-ead/logs"

Nous téléchargeons le modèle (à n'executer qu'une fois)

Nous l'importons maintenant dans la session avec une configuration spéciale (4bit, bf16) pour accélérer le fine-tuning



In [ ]:
# Load tokenizer and model with QLoRA configuration
# si message erreur : Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`,
# redémarrer le noyau et relancer l'exécution
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

Nous téléchargeons le "tokenizer" : c'est une manière de convertir le texte en une série d'identifiants qui prennent moins de place. Selon les cas, un mot peut être représenté par un seul token ou par plusieurs.

In [ ]:
!mkdir tokenizer && cd tokenizer

mkdir: cannot create directory ‘tokenizer’: File exists


In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    inference_mode=False,
    task_type="CAUSAL_LM",
    target_modules = ["q_proj", "v_proj"] #There are options to deepen the finetuning by unfreezing more weights but with a cost in performance
)

tokenizer = LlamaTokenizerFast.from_pretrained(model_name, add_eos_token=True, from_slow=True)
tokenizer.pad_token = tokenizer.eos_token

# This is the fix for fp16 training
tokenizer.padding_side = "right"

In [ ]:
!cd ..

**préparation des données**


On télécharge les données et on procède à la conversation dans un format ad hoc bien adapté à llama-base. La structure en dièse (###) est déjà reconnue.

À noter que nous n'utilisons pas le format chat (type [INST]).

In [ ]:
!pip install -q datasets

  Preparing metadata (setup.py) ... done


In [ ]:
from datasets import load_dataset
from random import randrange

In [ ]:
# Chargement du dataset.
# dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
data_files = {"train": "/content/drive/MyDrive/resultats_tableau_final.json"}
dataset = load_dataset("json", data_files=data_files, split="train")
print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

dataset size: 28639
{'answer': '<c level="recordgrp">\n<did>\n<unitid>1/M/167</unitid>\n<unitdate normal="1856-02-27">27/02/1856</unitdate>\n<unittitle>CORNEN, GABRIEL. Né le 27/02/1856 à PLOUARZEL (29). Matricule : 42759.</unittitle>\n</did>\n</c><c level="recordgrp">\n<did>\n<unitid>1/M/235</unitid>\n<unitdate normal="1879-06-06">06/06/1879</unitdate>\n<unittitle>CORNEN, GUILLAUME MARIE. Né le 06/06/1879 à LOCMARIA PLOUZANE (29). Matricule : 76530.</unittitle>\n</did>\n</c>', 'instruction': '1/M/16727/02/1856CORNEN, GABRIEL. Né le 27/02/1856 à PLOUARZEL (29). Matricule : 42759.1/M/23506/06/1879CORNEN, GUILLAUME MARIE. Né le 06/06/1879 à LOCMARIA PLOUZANE (29). Matricule : 76530.', 'ead_id': '10070_FRSHD_IND_1M.xml'}


In [ ]:
from random import randint
def format_alpaca(sample):
    instruction = f"### Instruction\n{sample['instruction']}"
    context = "### Contexte\nPourrais-tu m'encoder ce texte de description archivistique au format XML/EAD ? Attention, il ne s'agit que de balises <c> dans <dsc>."
    response = f"### Réponse\n{sample['answer']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [instruction, context, response] if i is not None])
    return prompt

def template_dataset(sample):
    sample["text"] = f"{format_alpaca(sample)}{tokenizer.eos_token}"
    return sample

dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
print(f"dataset size: {len(dataset)}")
print(dataset[randint(0, len(dataset))]["text"])

Map:   0%|          | 0/28639 [00:00<?, ? examples/s]

dataset size: 28639
### Instruction
1/M/3810/02/1804BEVEN, Yves. Né le 10/02/1804 à ILE DE GROIX (56). Matricule : 24032.1/M/4526/04/1814BEVER, Yves. Né le 26/04/1814 à PLEUBIAN (22). Matricule : 27667.

### Contexte
Pourrais-tu m'encoder ce texte de description archivistique au format XML/EAD ? Attention, il ne s'agit que de balises <c> dans <dsc>.

### Réponse
<c level="recordgrp">
<did>
<unitid>1/M/38</unitid>
<unitdate normal="1804-02-10">10/02/1804</unitdate>
<unittitle>BEVEN, Yves. Né le 10/02/1804 à ILE DE GROIX (56). Matricule : 24032.</unittitle>
</did>
</c><c level="recordgrp">
<did>
<unitid>1/M/45</unitid>
<unitdate normal="1814-04-26">26/04/1814</unitdate>
<unittitle>BEVER, Yves. Né le 26/04/1814 à PLEUBIAN (22). Matricule : 27667.</unittitle>
</did>
</c></s>


fine tuning llama2

In [ ]:
torch.cuda.empty_cache() #libérer la mémoire GPU

In [ ]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    auto_find_batch_size=auto_find_batch_size,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

trainer.train() #Pour un nouveau fine-tuning.
#trainer.train(resume_from_checkpoint=True) #Pour prolonger un fine-tuning existant.
trainer.model.save_pretrained(output_dir)

Nous sauvons le modèle

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(new_model_name)

On va maintenant fusionner le modèle et le LORA pour aller plus vite en inférence :

In [ ]:
!pip install -q accelerate -U

In [ ]:
del model
torch.cuda.empty_cache()

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
output_merged_dir = os.path.join(new_model_name, "final_merged_checkpoint")
model.save_pretrained(output_merged_dir, safe_serialization=True)

**Inférence**

Avant de lancer l'inférence, je recommande de vider la mémoire de la session (Runtime > Restart runtime)

Nous exportons les fichiers du tokenizer :

In [ ]:
!cp "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-hf/tokenizer.json" "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-ead-v5/final_merged_checkpoint/tokenizer.json"
!cp "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-hf/tokenizer.model" "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-ead-v5/final_merged_checkpoint/tokenizer.model"
!cp "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-hf/tokenizer_config.json" "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-ead-v5/final_merged_checkpoint/tokenizer_config.json"
!cp "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-hf/special_tokens_map.json" "/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-ead-v5/final_merged_checkpoint/special_tokens_map.json"

Pour l'inférence, on utilise vllm. Il est conseillé de redémarrer l'environnement d'exécution.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install https://github.com/vllm-project/vllm/releases/download/v0.2.2/vllm-0.2.2+cu118-cp310-cp310-manylinux1_x86_64.whl

In [ ]:
!pip uninstall torch -y
!pip install torch --upgrade --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install -U -q xformers --index-url https://download.pytorch.org/whl/cu118

In [ ]:
from vllm import LLM, SamplingParams
import xformers
import os

In [ ]:
llm = LLM("/content/drive/MyDrive/Automat_EAD/GT_EAD/llama-2-7b-ead-v4/final_merged_checkpoint")

In [ ]:
sampling_params = SamplingParams(temperature=0.9, top_p=0.95, max_tokens=100000)

In [ ]:
prompts = ["""### Instruction
I – CORRESPONDANCE GENERALE
1 – Courrier « arrivée » (1946-1954)
Courrier arrivée non protégé (1952 – 1954)
Art. 1

DCSSA (1952)

Art. 2

Service de santé : DSS Brest, DCSSA, établissement central de réanimation –
transfusion de l’armée. EMG / Service technique des machines. DCTIM.
CIRAM (1953)

Art. 3

Service de santé : DSS Brest, DCSSA. EMG / Service technique des
machines. DCTIM. CIRAM (1954)

Courrier arrivée classifié (1946-1954)
Art. 4

Répertoire du courrier arrivée classifié (1946-1953)

Art. 5

1946

Art. 6

1947

Art. 7

1948

Art. 8

1949

Art. 9

1950

Art. 10

1951

Art. 11

1952

Art. 12

1953

Art. 13

Bulletin d’information de la 2ème RM - Etat-major n°4 et n° 6 à 11 (mai –
décembre 1953)

Art. 14

1954

2 – Courrier « départ » (1947-1962)
Courrier « Départ » classifié (1947 – 1962)
Art. 15

Répertoire du courrier départ classifié (1947-1952)

Art. 16

1947

Art. 17

1948

5

Art. 18

1949

Art. 19

1950

Art. 20

1951

Art. 21

1952

Art. 22

1956

Art. 23

1957

Art. 24

1958

Art. 25

1959

Art. 26

1960

Art. 27

1961

Art. 28

1962 (dont inspection générale)

Courrier départ du Directeur du Port (1955)
Art.29

1955





### Contexte
Pourrais-tu m'encoder ce texte de description archivistique au format XML/EAD ?""",

"""### Instruction

II – ORGANISATION ET FONCTIONNEMENT DE LA
DIRECTION DU PORT DE BREST
1 – Activité et fonctionnement du Port de Brest (1952-1962)
Art. 30

Compte-rendu trimestriel d’activité (15 décembre 1952 – 15 mars 1953)

Art. 31

Compte-rendu semestriel d’activité (15 mars – 15 septembre 1954)

Art. 32

Situation générale au 1er avril 1954

Art. 33

Compte rendu semestriel d’activité (15 mars – 15 septembre 1955)

Art. 34

Comptes-rendus sur l’état des bases (1956-1962)

Art. 35

Rapport sur le fonctionnement des services (1er octobre 1957 – 15 juillet
1958)

Art. 36

Rapport sur le fonctionnement des services (15 juillet 1958- 1er mai 1959)

Art. 37

Rapport sur le fonctionnement des services (1er mai 1959 – 1er mai 1960)

Art. 38

Rapport sur le fonctionnement des services (1er mai 1960 - 1er mai 1961

Art. 39

Rapport sur le fonctionnement des services (1er mai 1961 - 1er mai 1962)

6

2 – Rapports sur le fonctionnement des bâtiments relevant de
la Direction du Port (1957-1962)
Art. 40

RFS Rhinocéros, Eléphant, Implacable, Imbattable, Acharné, Locminé,
Cigale, Falleron. (1er oct. 1957 – 15 juil. 1958)

Art. 41

RFS Rhinocéros, Implacable, Imbattable, Acharné, Entêté, Locminé, Cigale.
(15 juillet 1958 - 1er mai 1959)

Art. 42

RFS Rhinocéros, Eléphant, Implacable, Imbattable, Entêté, Hercule, Cigale,
Issole, Falleron (1er mai 1959 – 1er mai 1960)

Art. 43

RFS Rhinocéros, Eléphant, Implacable, Imbattable, Acharné, Entêté,
Hercule, Cigale, Issole, Falleron (1er mai 1960 - 1er mai 1961)

Art. 44

RFS Rhinocéros, Eléphant, Implacable, Imbattable, Acharné, Entêté,
Hercule, Cigale, Issole, Falleron (1er mai 1961 - 1er mai 1962)

3 – Rapports. Comptes rendus (1956-1962)
Art. 45

Rapports sur le moral du personnel (1956-1958)

Art. 46

Comptes rendus de mission dont : projet de gabare du port, ravitaillement en
eau des îles de Molène et de Sein (1958-1959). Voyage de Saïgon à Djibouti
du remorqueur « Coolie » de Marine à Madagascar (1956). CR d’avarie du
pousseur « Palétuvier » (10 octobre 1958)

Art. 47

Comptes rendus d’exercice : Castinets, Nemo, Grande chance, Marfa, Semicolonne, Misaine, Artimon, Loutre bleue, Ship shape, Triton, Sterne,
Akward, Side Step, Fallex, Oscar, Brestex, Dawn Breeze, Lordra, Bresdra,
Submiss, Cherdragon (1952-1962)

### Contexte
Pourrais-tu m'encoder ce texte de description archivistique au format XML/EAD ?"""]

In [ ]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:57<00:00, 57.84s/it]


In [ ]:
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Generated text: {generated_text!r}")

Generated text: '\n\n### Réponse\n<c level="recordgrp">\n<did>\n<unittitle>Courrier arrivée non protégé (1952 – 1954)</unittitle>\n<physdesc>Art. 1</physdesc>\n</did>\n<controlaccess>\n<persname role="nom">DCSSA (1952)</persname>\n</controlaccess>\n<physdesc>Art. 2</physdesc>\n</c><c level="recordgrp">\n<did>\n<unittitle>Art. 3</unittitle>\n<physdesc>Art. 3</physdesc>\n</did>\n<controlaccess>\n<persname role="nom">Service de santé : DSS Brest, DCSSA. EMG / Service technique des machines. DCTIM. CIRAM (1954)</persname>\n</controlaccess>\n</c>\n### Réponse\n<c level="recordgrp">\n<did>\n<unittitle>Art. 1</unittitle>\n<physdesc>Art. 1</physdesc>\n</did>\n<controlaccess>\n<persname role="nom">DCSSA (1952)</persname>\n</controlaccess>\n</c>\n</persname>\n<persname role="nom">Service de santé : DSS Brest, DCSSA. EMG / Service technique des machines. DCTIM. CIRAM (1954)</persname>\n</c><c level="recordgrp">\n<did>\n<unittitle>Art. 2</unittitle>\n<physdesc>Art. 2</physdesc>\n</did>\n<controlac